In [46]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc

In [22]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

from oclr import OneCycleLR, LRFinder

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [3]:
df_train = pd.read_csv("price_prediction/train.csv")
df_test = pd.read_csv("price_prediction/test.csv")

In [4]:
df_train['text'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text']))
df_train['text_encoded'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text_encoded']))
df_train['char_encoded'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['char_encoded']))

df_test['text'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text']))
df_test['text_encoded'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text_encoded']))
df_test['char_encoded'] = Parallel(n_jobs=20, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['char_encoded']))


# GL encodings

In [5]:
from data_science_utils.preprocessing import NeuralCategoricalFeatureTransformer

ct_nn = NeuralCategoricalFeatureTransformer(cols=["GL"],prefix="gl_encoded_",
                                            target_columns=["PRICE"],verbose=1,n_components=16,n_iter=200,)

ct_nn.fit(df_train)

ct_nn.skip_fit = True

Neural Categorical fit start at: 2019-06-13 08:02:33.734957
Instructions for updating:
Colocations handled automatically by placer.


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Shape of Input to Neural Network: (61, 61), Output shape: (61, 69)
Instructions for updating:
Use tf.cast instead.
Train on 37 samples, validate on 24 samples
Epoch 1/200
37/37 [==============================] - 2s 57ms/step - loss: 0.2424 - val_loss: 0.2383
Epoch 2/200
37/37 [==============================] - 0s 53us/step - loss: 0.2370 - val_loss: 0.2334
Epoch 3/200
37/37 [==============================] - 0s 43us/step - loss: 0.2301 - val_loss: 0.2262
Epoch 4/200
37/37 [==============================] - 0s 42us/step - loss: 0.2196 - val_loss: 0.2160
Epoch 5/200
37/37 [==============================] - 0s 66us/step - loss: 0.2048 - val_loss: 0.2027
Epoch 6/200
37/37 [==============================] - 0s 40us/step - loss: 0.1854 - val_loss: 0.1862
Epoch 7/200
37/37 [==============================] - 0s 44us/step - loss: 0.1619 - val_loss: 0.1667
Epoch 8/200
37/37 [==============================] - 0s 40us/step - loss: 0.1354 - val_loss: 0.1449
Epoch 9/200
37/37 [======================

In [6]:
df_train = ct_nn.transform(df_train)
df_test = ct_nn.transform(df_test)

In [7]:
gl_cols = get_specific_cols(df_train,prefix='gl_')

# Pretrained Embeddings Model

In [8]:
start = time()
glove = api.load("glove-twitter-25") 
print("total = ",(time()-start))




/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


total =  91.9950692653656


In [24]:
ptr_glove = PreTrainedEmbeddingsTransformer(glove,size=25)
ptr_glove.fit()

In [25]:

df_test['glove_encoded'] = ptr_glove.transform(df_test['text'].values)
df_train['glove_encoded'] = ptr_glove.transform(df_train['text'].values)


Fasttext Transforms start at: 2019-06-13 08:57:47.014103
Number of Unique Test Tokens for Fasttext transform 160522



Fasttext Transforms done at: 2019-06-13 08:57:56.228158
Fasttext Transforms start at: 2019-06-13 08:57:56.476962
Number of Unique Test Tokens for Fasttext transform 332242



Fasttext Transforms done at: 2019-06-13 08:58:29.763856


In [ ]:
start = time()
glove = api.load("glove-twitter-50") 
print("total = ",(time()-start))

ptr_glove_50 = PreTrainedEmbeddingsTransformer(glove,size=50)
ptr_glove_50.fit()

df_test['glove_encoded-50'] = ptr_glove_50.transform(df_test['text'].values)
df_train['glove_encoded-50'] = ptr_glove_50.transform(df_train['text'].values)


In [ ]:
start = time()
fasttext = api.load("fasttext-wiki-news-subwords-300") 
print("total = ",(time()-start))

ptr = PreTrainedEmbeddingsTransformer(fasttext,size=300)
ptr.fit()






In [ ]:
df_train['fasttext_encoded'] = ptr.transform(df_train['text'].values)
df_test['fasttext_encoded'] = ptr.transform(df_test['text'].values)

In [26]:
maxlen = 100
batch_size = 4096
embedding_dims = 25
epochs = 5


X,X_gl,y = df_train['glove_encoded'].values,df_train[gl_cols],df_train['PRICE'].values
x_train, x_test,x_gl_train,x_gl_test, y_train, y_test = train_test_split(X,X_gl, y, test_size=0.2, random_state=42)


x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


In [35]:
print(x_train.shape)

x_train = x_train.reshape((-1,100,25,1))
x_test = x_test.reshape((-1,100,25,1))
print(x_train.shape)

(1159686, 100, 25)
(1159686, 100, 25, 1)


In [42]:
gc.collect()

1157

In [ ]:
epochs = 20

datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,)
datagen.fit(x_train)
train_iterator = datagen.flow((x_train,x_gl_train), y_train, batch_size=2048,shuffle=True)
validation_iterator = datagen.flow((x_test,x_gl_test), y_test, batch_size=2048,shuffle=True)



main_input = Input(shape=(None,embedding_dims,1), dtype='float32', name='main_input')
x = Lambda(lambda x:K.reshape(x,(-1,100,25)))(main_input)
x = conv_layer(x,n_kernels=32,kernel_size=3,padding='valid')
x1 = MaxPooling1D()(x)
x2 = AveragePooling1D()(x)
x = concatenate([x1,x2])
x = transition_layer(x, n_kernels=32,dropout=0)

xp = x
x = conv_layer(x,n_kernels=64,kernel_size=3,padding='same')
x = transition_layer(x, n_kernels=32,dropout=0)

xp2 = x
x = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=2,padding='same')
x = transition_layer(x, n_kernels=32,dropout=0)
x = concatenate([x,xp,xp2])
x = transition_layer(x, n_kernels=64,dropout=0)
x = pre_dense_layer(x)
K.int_shape(x)
auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x_aux = Dense(4)(auxiliary_input)
x = concatenate([x,x_aux])
x = Dense(16)(x)
x = Dropout(0.2)(x)
x =Activation('relu')(x)

main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

optimizer = Adam(lr=0.0001,)

model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
print("params = ",model.count_params())

checkpoint = ModelCheckpoint("model.hdf5", monitor='mse', verbose=0, save_best_only=True, mode='max')
callbacks_list = []

# model.fit([x_train,x_gl_train], y_train,
#           batch_size=batch_size,
#           epochs=epochs,callbacks=callbacks_list,
#           validation_data=([x_test,x_gl_test], y_test))


model.fit_generator(train_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_data = validation_iterator, 
                    validation_steps = len(validation_iterator),
                    epochs=epochs, verbose=1,
                    callbacks=callbacks_list,shuffle=True)



```
params =  101633
Epoch 1/5
567/567 [==============================] - 364s 642ms/step - loss: 4031846918.7346 - mean_absolute_error: 1066.6701 - mean_squared_error: 4031846918.7346 - val_loss: 20270204.2412 - val_mean_absolute_error: 963.9843 - val_mean_squared_error: 20270204.2412
 - lr: 0.00001 
Epoch 3/5
567/567 [==============================] - 360s 635ms/step - loss: 4031529518.1321 - mean_absolute_error: 983.9032 - mean_squared_error: 4031529518.1321 - val_loss: 19970991.7471 - val_mean_absolute_error: 886.9307 - val_mean_squared_error: 19970991.7471
 - lr: 0.00001 
Epoch 4/5
567/567 [==============================] - 359s 633ms/step - loss: 4031241021.6406 - mean_absolute_error: 924.6773 - mean_squared_error: 4031241021.6406 - val_loss: 19779274.3431 - val_mean_absolute_error: 851.5251 - val_mean_squared_error: 19779274.3431
 - lr: 0.00001 
Epoch 5/5
567/567 [==============================] - 360s 635ms/step - loss: 4030976892.7566 - mean_absolute_error: 897.6240 - mean_squared_error: 4030976892.7566 - val_loss: 19615373.1328 - val_mean_absolute_error: 831.4093 - val_mean_squared_error: 19615373.1328
 - lr: 0.00001 
```

In [ ]:
epochs = 20

datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,)
datagen.fit(x_train)
train_iterator = datagen.flow((x_train,x_gl_train), y_train, batch_size=2048,shuffle=True)
validation_iterator = datagen.flow((x_test,x_gl_test), y_test, batch_size=2048,shuffle=True)



main_input = Input(shape=(None,embedding_dims,1), dtype='float32', name='main_input')
x = Lambda(lambda x:K.reshape(x,(-1,100,25)))(main_input)
x = conv_layer(x,n_kernels=64,kernel_size=3,padding='valid')
x1 = MaxPooling1D(pool_size=2)(x)
x2 = AveragePooling1D(pool_size=2)(x)
x = concatenate([x1,x2])
xp = x
x = transition_layer(x, n_kernels=32,dropout=0)
x = conv_layer(x,n_kernels=64,kernel_size=3,padding='same')
x = transition_layer(x, n_kernels=32,dropout=0)

xp2 = x
x = concatenate([x,xp])
x = transition_layer(x, n_kernels=32,dropout=0)
x = conv_layer(x,n_kernels=64,kernel_size=3,dilation_rate=2,padding='same')
x = transition_layer(x, n_kernels=32,dropout=0)

x = concatenate([x,xp,xp2])

x = transition_layer(x, n_kernels=128,dropout=0)
x = pre_dense_layer(x)
K.int_shape(x)
auxiliary_input =  Input(shape=(len(gl_cols),), dtype='float32', name='aux_input')
x_aux = Dense(4)(auxiliary_input)
x = concatenate([x,x_aux])
x = Dense(32)(x)
x = Dropout(0.1)(x)
x =Activation('relu')(x)

main_output = Dense(1)(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

optimizer = Adam(lr=0.0001,)

model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mae','mse'])
print("params = ",model.count_params())

checkpoint = ModelCheckpoint("model.hdf5", monitor='mse', verbose=0, save_best_only=True, mode='max')
callbacks_list = []

# model.fit([x_train,x_gl_train], y_train,
#           batch_size=batch_size,
#           epochs=epochs,callbacks=callbacks_list,
#           validation_data=([x_test,x_gl_test], y_test))


model.fit_generator(train_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_data = validation_iterator, 
                    validation_steps = len(validation_iterator),
                    epochs=epochs, verbose=1,
                    callbacks=callbacks_list,shuffle=True)

